# First look at `rootpy`

(**note** a lot of this tutorial draws comparisons with the pyROOT tutorial in this same repository, This notebook follows very closely the same steps done in the pyROOT firstLook notebook)

The [rootpy](http://www.rootpy.org/) homepage describes itself as:

```
The rootpy project is a community-driven initiative aiming to provide a more pythonic interface with ROOT on top of the existing PyROOT bindings. Given Python’s reflective and dynamic nature, rootpy also aims to improve ROOT design flaws and supplement existing ROOT functionality. 
```

The first thing we'll do is load a file via XRootD. The pyROOT code to load a file is
```python
import ROOT as r
firstLookFile = r.TFile.Open("root://cmseos.fnal.gov//store/user/hats/PyRoot/2017/qcd_samples/shuffled_sig/sig_shuffled_0.root")
```

In this snippet, `ROOT`, `TFile` and `TFile.Open` are direct mappings to the underlying ROOT libraries. In `pyROOT`, you have to explicitly burrow down through the `ROOT` object system.

By comparison, the `rootpy` library makes user code more pythonic. It provides a lot of easy interfaces to simplify regular operations. The `pyROOT` code from before can be implemented as the following with `pyroot`:

In [ ]:
from rootpy.io import root_open
firstLookFile = root_open('root://cmseos.fnal.gov//store/user/hats/PyRoot/2017/qcd_samples/shuffled_sig/sig_shuffled_0.root')
firstLookFile.ls()

In `pyROOT`, you have to explicitly call `ROOT->TFile->Open`, just to load a file. On the other hand, `rootpy` can divine out that receiving a cardboard box requires a similarly sized back at the same time

# Why `rootpy`?
Rootpy extends the ROOT classes providing new functionalities, this means that the old ROOT methods are still available, but new ones are provided to make your life easier. For example you can acess the rootfile content with the attribute getter (the dot).

If you recall, retrieving a `TTree` from a `TFile`, then printing the branch names with `pyROOT` required the following snippet:
```python
firstLookTree = firstLookFile.Get("tree")
for branch in firstLookTree.GetListOfBranches():
    print branch.GetName()
```

Let's look at the equivalent code in `rootpy`:

In [ ]:
hatsTree = firstLookFile.tree
for branch in hatsTree.branches:
  print branch.GetName()

`Rootpy` helps simplify your code by adding additional members and functionality to the "bare" `pyROOT` objects. In this example, we can see that the "`.`" operator is overridden for `TFile` and `TTree` objects to handle getting children and the list of branches.

You have to be careful -- `rootpy` does not wrap every single ROOT class. Some omissions are design choices, and others are just too hard. Also, using a bare ROOT method, will return a ROOT object and not a `rootpy` one, hence you might want to wrap the the method call with ```asrootpy()``` to convert it to a `rootpy` object (we will see an example later)

Unlike ROOT or `pyROOT`, in `rootpy` there is no need to define the name and the title of the objects, unless you want to write them out to files.

In [ ]:
from rootpy.plotting import Canvas
# No need for useless names and titles, but you can give them if you need!
canvas = Canvas(800, 800)

The Draw command of the `TTree` is overridden to allow a more pythonic interface (we will see it later). To check how it works, simply type:

In [ ]:
help(hatsTree.Draw)

One nice feature is that `Draw()` returns the histogram created, so we can play with it later on. 

In [ ]:
histo = hatsTree.Draw('jetAK4_pt')
# Required to show the plot inline in Jupyter
canvas.Draw()

Let's update this plot to be a bit prettier. First, we need to change a global ROOT option

In [ ]:
import ROOT
ROOT.gStyle.SetOptStat(0)

Styling histograms in ROOT can be a significant pain and waste of time. `Rootpy` tries to simplify the whole process by providing simple access to all the drawing parameters. Additionally, it extends the definition to matplotlib-compatible markers, lines and colors. **Web colors (hexadecimal format) are also available!**

In [ ]:
histo.markersize = 1
histo.markerstyle = 20
histo.markercolor = '#336666' #supports matplotlib, hex and ROOT colors!
histo.xaxis.title = 'p_{T} [GeV]'
histo.yaxis.title = 'Entries'
histo.title = 'Some nice title'
histo.Draw()
canvas.Draw()

Histograms are iterable in `rootpy`. This means you can loop over the bins like you would loop over a list
```python
for hbin in histogram:
  #do things
  pass
```

Each bins contains information about the position (accesses through .x, .y, .z), value, error, and if it's an overflow. The content can be accessessed and written, making bin-by-bin calculations way easier.

In [ ]:
maxbin = max(histo, key = lambda x: x.value) #the histogram is iterable
print 'Maximum bin information:'
print '  # entries:     ', maxbin.value
print '  uncertainty:   ', maxbin.error
print '  x value:       ', maxbin.x.center
print '  x bin low edge:', maxbin.x.low